# BERT를 이용한 Chatbot 만들기
> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)


이번 과제의 목표는 BERT를 사용해 간단한 Chatbot을 어떻게 만들지 고민해 봅니다.

아래 실습 이후, 과제를 위한 챗봇 데이터는 다음의 링크에서 받아서 활용해주세요.   
https://github.com/songys/Chatbot_data   

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 8.0MB/s 
     |████████████████████████████████| 890kB 39.6MB/s 
     |████████████████████████████████| 3.2MB 25.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ab98653f9287c3b68a989b1f00941407900f4292850e215088caf7b4ebf02ee7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## Pretrained BERT의  [CLS] token을 이용한 Chatbot.
### 사전 준비물
1. Pretrain된 BERT 모델.
2. 질의응답 Dataset.

### 진행 과정
1. 사용자의 질문(query)를 입력 받는다.
2. query를 pretrained BERT의 입력으로 넣어, query 문장에 해당하는 [CLS] token hidden을 얻는다.
3. 사전에 준비된 질의응답 Dataset에 존재하는 모든 질문들을 pretrained BERT의 입력으로 넣어, 질문들에 해당하는 [CLS] token hidden을 얻는다.
4. query의 [CLS] token hidden과 질문들의 [CLS] token hidden간의 코사인 유사도를 구한다.
5. 가장 높은 코사인 유사도를 가진 질문의 답변을 반환시켜준다.
6. 위 과정 반복.

### Pretrained BERT Load
* 공개된 BERT base Multilingual 버전을 사용한다.

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.parameters

<bound method Module.parameters of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Drop

### 질의응답 Dataset 예시
* 같은 index가 서로 pair 입니다.

In [ ]:
chatbot_Question = ['기차 타고 여행 가고 싶어','꿈이 이루어질까?','내년에는 더 행복해질려고 이렇게 힘든가봅니다', '간만에 휴식 중', '오늘도 힘차게!'] # 질문
chatbot_Answer = ['꿈꾸던 여행이네요.','현실을 꿈처럼 만들어봐요.','더 행복해질 거예요.', '휴식도 필요하죠', '아자아자 화이팅!!'] # 답변
print(chatbot_Question[:])
print(chatbot_Answer[:])

['기차 타고 여행 가고 싶어', '꿈이 이루어질까?', '내년에는 더 행복해질려고 이렇게 힘든가봅니다', '간만에 휴식 중', '오늘도 힘차게!']
['꿈꾸던 여행이네요.', '현실을 꿈처럼 만들어봐요.', '더 행복해질 거예요.', '휴식도 필요하죠', '아자아자 화이팅!!']


### [CLS] token을 얻기 위한 함수

In [ ]:
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(    # **tokenized_sent
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

* query [CLS] token hidden 확인

In [ ]:
query = '아 여행가고 싶다~'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape)

[[ 1.10308483e-01 -1.02158368e-01  2.36617401e-01  5.52501418e-02
   3.93724412e-01  2.43569091e-01  1.86358243e-02 -2.02085018e-01
  -9.63337943e-02  3.54374230e-01 -7.72638470e-02  3.26404721e-01
  -1.74165383e-01  6.97085708e-02 -5.34738004e-02 -1.16455317e-01
  -2.62405455e-01  1.22647405e-01  5.06818235e-01 -9.67907086e-02
   1.23077102e-01  7.92990625e-02 -2.98088845e-02  3.16682041e-01
  -1.79102309e-02  2.30732054e-01  3.72239828e-01 -1.27580717e-01
   5.79192102e-01  4.66822356e-01  4.10569042e-01  1.01960190e-01
  -2.56263286e-01  2.49144495e-01  4.32021737e-01  1.58709645e-01
  -1.90560234e+00  1.18138477e-01 -2.43944693e-02 -7.49023929e-02
  -5.16358435e-01  1.96145207e-01  3.76856387e-01  1.58740014e-01
   5.48579574e-01  1.11865151e+00  6.56481206e-01  8.93770382e-02
   1.43516183e+00 -5.34082055e-01  3.55657727e-01 -6.39794886e-01
  -5.98752163e-02 -1.56276774e+00  2.52752364e-01  1.68553054e-01
   3.63084167e-01 -1.66097155e-03  9.51945111e-02 -1.09238543e-01
  -3.28798

* Chatbot 데이터셋의 질문 [CLS] token hidden 확인

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

dataset_cls_hidden = []
for q in chatbot_Question:
    q_cls = get_cls_token(q)
    dataset_cls_hidden.append(q_cls)
dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)
print(dataset_cls_hidden)   # 데이터셋의 질문에 대한 [CLS] 토큰 벡터
print(dataset_cls_hidden.shape)


[[ 0.2015294   0.00450035  0.19752151 ...  0.54929185  0.02755525
   0.21813345]
 [-0.09731648  0.10722532 -0.23476414 ...  0.36494994  0.454968
   0.31987903]
 [-0.23625095  0.13388395 -0.28571615 ...  0.53231657  0.33486852
   0.30000678]
 [ 0.02712039  0.12172284  0.07334824 ...  0.23747024  0.2893383
  -0.30687687]
 [-0.2990097  -0.0900768   0.10507746 ...  0.3788135   0.38025463
   0.30487213]]
(5, 768)


### 코사인 유사도

In [ ]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)   # 데이터셋의 0번째 질문과 가장 유사하군요!
print(cos_sim)


[[0.85016316 0.7788855  0.73615134 0.7798742  0.72420174]]


* Chatbot 데이터 셋중 가장 유사도가 높은 질문 선택 및 답변

In [ ]:
top_question = np.argmax(cos_sim)

print('나의 질문: ', query)
print('저장된 답변: ', chatbot_Answer[top_question])

나의 질문:  아 여행가고 싶다~
저장된 답변:  꿈꾸던 여행이네요.
